### This notebook will contain the loading of the datasets, and tie together the different parts of the project

In [1]:
from DataLoader import DataLoader
from models.CNN import *
from models.FFN import *
import torch
import torch.nn as nn
from models.CointegrationResidualGenerator import CointegrationResidualGenerator
from models.BacktestSharpeEvaluator import BacktestSharpeEvaluator
from PortfolioOptimizer import PortfolioOptimizer



Data loaded successfully from ../data/european_wholesale_electricity_price_data_daily.csv

--- Germany 2020 Data ---
               Country       Date   Price
50037          Austria 2020-01-01   32.14
50038          Belgium 2020-01-01   35.18
50039         Bulgaria 2020-01-01   31.24
50040          Croatia 2020-01-01   34.99
50041          Czechia 2020-01-01   32.07
50042          Denmark 2020-01-01   30.88
50043          Estonia 2020-01-01   27.04
50044          Finland 2020-01-01   27.04
50045           France 2020-01-01   34.94
50046          Germany 2020-01-01   31.48
50047           Greece 2020-01-01   52.26
50048          Hungary 2020-01-01   32.07
50049          Ireland 2020-01-01   47.52
50050            Italy 2020-01-01   35.53
50051           Latvia 2020-01-01   27.04
50052        Lithuania 2020-01-01   27.04
50053       Luxembourg 2020-01-01   31.48
50054       Montenegro 2020-01-01  106.38
50055      Netherlands 2020-01-01   34.31
50056  North Macedonia 2020-01-01  104.00
S

## Sharpe ratio

In [2]:
def sharpe_ratio_loss(returns, risk_free_rate=0.0):
    """
    Custom loss function to maximize the Sharpe Ratio.
    Args:
        returns (torch.Tensor): Predicted returns
        risk_free_rate (float): Risk-free rate for Sharpe calculation
    Returns:
        torch.Tensor: Negative Sharpe Ratio (to minimize)
    """
    excess_returns = returns - risk_free_rate
    mean_excess = torch.mean(excess_returns)
    std_excess = torch.std(excess_returns, unbiased=False) + 1e-6  # epsilon for stability
    sharpe_ratio = mean_excess / std_excess
    return -sharpe_ratio

In [3]:

# MAIN FUNCTION
### -------------- PARSING DATA -------------- ###
parser = DataLoader(file_path="../data/european_wholesale_electricity_price_data_daily.csv")

# Get list of countries
print("\n--- List of Countries ---")
all_countries_list = parser.get_country_list()
print(all_countries_list)

# Get daily price matrix for all countries for the entire year 2021
price_matrix = parser.get_price_matrix(
    time_range="2021-01-01,2021-12-31",
    countries=all_countries_list,
    fill_method="ffill"
)

# Get the raw daily returns for the price matrix
returns = price_matrix.pct_change().dropna()



### -------------- COINTEGRATION RESIDUALS -------------- ###
# Create an instance of the CointegrationResidualGenerator
residual_generator = CointegrationResidualGenerator(price_matrix)

residual_generator.compute_all_asset_residuals()

# Get residuals
asset_residuals = residual_generator.get_asset_residuals()

# Get the input for CNN
# cnn_input contains a set of 329 data samples, each sample represents 30-day cumulative residuals for the 31 countries
cumulative_residual_window = 30
cnn_input = residual_generator.prepare_cnn_input_from_residuals(window=cumulative_residual_window)

# Get the start index of the first 30-day cumulative residuals in the returns DataFrame
start_idx_in_returns = returns.index.get_loc(asset_residuals.index[0])
num_samples = len(asset_residuals) - cumulative_residual_window + 1
next_day_indices = [start_idx_in_returns + i + cumulative_residual_window for i in range(num_samples)]

# Get the next-day returns for the corresponding indices
# The next-day returns are the returns for the day after the last day of each 30-day window
# For example, if the first 30-day window ends on index 0, the next day return is at index 1
# If the second 30-day window ends on index 1, the next day return is at index 2, and so on.
next_day_returns = returns.iloc[next_day_indices]




### -------------- FEED THE 30-DAY CUMULATIVE RESIDUALS OF EVERY COUNTRY TO CNN+FFN -------------- ###
# Transform cnn_input to be compatible with the CNN input shape
cnn_input_array = cnn_input.transpose(0, 2, 1) # [samples, features, window]

# FILIP'S SECTION: CNN+FFN
# Hey Filip, this is the section where you can add your code to train the CNN+FFN model.
# cnn_input_array essentially contains 329 training data points, each data point is 30-day cumulative residuals for the 31 countries.
# So, for one "data point" you would feed the set of 30-day cumulative residuals for every country to the CNN+FFN model.
# One data point should result in a set of weights for each of the 31 countries.
# Each set of weights is used to calculate one next-day portfolio return.
# Repeat this for all 329 data points to get a set of 329 portfolio returns.
# You can then use these portfolio returns to calculate the Sharpe ratio
# Optimize the CNN+FFN to maximize the Sharpe ratio.

# Set the device to GPU if available, otherwise MPS (for Mac silicon) or CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

torch.manual_seed(1)  # For reproducibility

# Create the PortfolioOptimizer instance and train the model
# NOTE: We need to have some validation data to evaluate the model performance
# otherwise we will be overfitting the model to the training data

# Grid search for hyperparameters
num_filters = [8, 16, 32]
filter_sizes = [3, 5, 7]
hidden_dims = [64, 128]

# Choose the best hyperparameters based on validation performance
best_score = float('-inf')
best_params = None

for num_filter in num_filters:
    for filter_size in filter_sizes:
        for hidden_dim in hidden_dims:
                print(f"Training with filters: {num_filter}, filter size: {filter_size}, hidden dim: {hidden_dim}")
                
                # Initialize the PortfolioOptimizer
                optimizer = PortfolioOptimizer(cnn_input_array, 
                                               next_day_returns, 
                                               batch_size=1000, num_epochs=1000, 
                                               num_filters=num_filter, 
                                               device=device, 
                                               filter_size=filter_size,
                                               hidden_dim=hidden_dim)
                
                # Train the model
                final_sharp, portfolio_returns = optimizer.train(verbose=False) # The final sharpe should be from the validation set

                if final_sharp > best_score:
                    best_score = final_sharp
                    best_params = (num_filter, filter_size, hidden_dim)
                    best_portfolio_returns = portfolio_returns
    
print(f"Best parameters: {best_params} with Sharpe Ratio: {best_score}")

### -------------- GET THE WEIGHTS OUTPUTTED FROM CNN+FFN -------------- ###
# Initializing the Sharpe ratio evaluator
#evaluator = BacktestSharpeEvaluator()

# Get the weight outputted from the CNN+FFN model
# One weight for each country
#weights = np.array([]) # CHANGE THIS TO THE ACTUAL WEIGHTS OUTPUTTED FROM THE CNN+FFN MODEL

# Normalize the weights using L1 normalization
# This is done to ensure that the portfolio is dollar-neutral
#normalized_weights = evaluator.normalize_weights_l1(weights)

# Multiply the normalized weights (vector) with the next-day returns (vector) to get the portfolio return
#next_day_portfolio_return = evaluator.compute_portfolio_return(normalized_weights, next_day_returns.iloc[0].values)

# Store the next day portfolio return 
#evaluator.add_return(next_day_portfolio_return)

# Repeat the above step for all 329 data points, adding the portfolio returns to the evaluator for each data point

# Once all portfolio returns are calculated, you can calculate the Sharpe ratio

# Train the model to optimize the Sharpe Ratio


Data loaded successfully from ../data/european_wholesale_electricity_price_data_daily.csv

--- List of Countries ---
['Austria', 'Belgium', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'Bulgaria', 'Serbia', 'Croatia', 'Montenegro', 'North Macedonia', 'Ireland']
Training with filters: 8, filter size: 3, hidden dim: 64


/Users/filip/stat7007/project/src/DataLoader.py:143: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_matrix = price_matrix.fillna(method=fill_method)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)



Final Sharpe Ratio: 1.8367
Training with filters: 8, filter size: 3, hidden dim: 128

Final Sharpe Ratio: 2.6306
Training with filters: 8, filter size: 5, hidden dim: 64

Final Sharpe Ratio: 1.5598
Training with filters: 8, filter size: 5, hidden dim: 128

Final Sharpe Ratio: 1.8720
Training with filters: 8, filter size: 7, hidden dim: 64

Final Sharpe Ratio: 1.0245
Training with filters: 8, filter size: 7, hidden dim: 128

Final Sharpe Ratio: 2.0866
Training with filters: 16, filter size: 3, hidden dim: 64

Final Sharpe Ratio: 1.7386
Training with filters: 16, filter size: 3, hidden dim: 128

Final Sharpe Ratio: 4.1862
Training with filters: 16, filter size: 5, hidden dim: 64

Final Sharpe Ratio: 1.1209
Training with filters: 16, filter size: 5, hidden dim: 128

Final Sharpe Ratio: 2.0067
Training with filters: 16, filter size: 7, hidden dim: 64

Final Sharpe Ratio: 1.2101
Training with filters: 16, filter size: 7, hidden dim: 128

Final Sharpe Ratio: 1.0992
Training with filters: 32